In [10]:
import os
import oci
from oci.generative_ai_inference.models import ChatDetails, CohereChatRequest, OnDemandServingMode, CohereTool, CohereParameterDefinition
import requests
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OCI_CONFIG_FILE = "~/.oci/config"
OCI_COMPARTMENT_ID = os.getenv("OCI_COMPARTMENT_ID")
config = oci.config.from_file(file_location=OCI_CONFIG_FILE, profile_name="CHICAGO")
generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(config)
model = "cohere.command-r-plus"

ImportError: cannot import name 'CohereTool' from 'oci.generative_ai_inference.models' (/home/ubuntu/.venv/lib/python3.11/site-packages/oci/generative_ai_inference/models/__init__.py)

## Employee Search

In [6]:
employee_id = {
    '田中 太郎': '12340001',
    '山田 花子': '12340002',
    '鈴木 次郎': '12340003',
    '佐藤 三郎': '56780001',
    '高橋 四郎':'56780004',
}

employee_detail = {
    '12340001': {'salary': '¥500', 'department': 'Sales'},
    '12340002': {'salary': '¥1,000', 'department': 'Engineer'},
    '12340003': {'salary': '¥300', 'department': 'Engineer'},
    '56780001': {'salary': '¥1,200', 'department': 'Sales'},
    '56780004': {'salary': '¥500', 'department': 'Marketing'},
}

department_detail = {
    'Sales': {'manager': '56780001', 'member_num': 2},
    'Engineer': {'manager': '12340002', 'member_num': 2},
    'Marketing': {'manager': '56780004', 'member_num': 1},
}

In [7]:
def get_employee_id(employee_name: str) -> dict:
    return {"employee_name": employee_name, "employee_id": employee_id[employee_name]}

def get_employee_detail(employee_id: str) -> dict:
    return {"employee_id": employee_id, "employee_detail": employee_detail[employee_id]}

def get_department_detail(employee_department: str) -> dict:
    return {"employee_department": employee_department, "employee_manager_id": department_detail[employee_department]}

In [8]:
tools=[CohereTool(
    name="get_employee_id",
    description="名前(employee_name)から社員ID(employee_id)を取得する関数",
    parameter_definitions={
            "employee_name": CohereParameterDefinition(
                description="社員名から社員IDを取得する。例: 田中太郎",
                type="str",
                is_required=True
            )
        }
    ),
    CohereTool(
    name="get_employee_detail",
    description="社員ID(employee_id)から収入(salary)や部門(department)を取得する関数",
    parameter_definitions={
            "employee_id": CohereParameterDefinition(
                description="社員IDから社員詳細情報を取得する。社員IDは[0-9]{8}で指定する。例: 12345678",
                type="str",
                is_required=True
            )
        }
    ),
    CohereTool(
    name="get_department_detail",
    description="部門(department)から部門のマネージャー(manager)や部門の人数(member_num)を取得する関数",
    parameter_definitions={
            "employee_department": CohereParameterDefinition(
                description="社員詳細情報から部署のマネージャーを取得する。例: Sales",
                type="str",
                is_required=True
            )
        }
    ),
]

tool_to_function_map = {
    "get_employee_id": get_employee_id,
    "get_employee_detail": get_employee_detail,
    "get_employee_manager": get_department_detail,
}

NameError: name 'CohereTool' is not defined

In [7]:
preamble = """
## 回答のスタイル
日本語で回答してください。
質問に対してできる限り詳細な回答をしてください。
"""
input_text = "田中 太郎のマネージャーを調べて"

In [8]:
chat_detail = ChatDetails(
    chat_request=CohereChatRequest(
        preamble_override=preamble,
        message=input_text,
        tools=tools,
        max_tokens=500,
        is_force_single_step=False,
        ),
    compartment_id=OCI_COMPARTMENT_ID,
    serving_mode=OnDemandServingMode(
        model_id=model
    ))
chat_response = generative_ai_inference_client.chat(chat_detail)
res_chat = chat_response.data.chat_response

while res_chat.tool_calls:
    print(f"tool_calls: {res_chat.tool_calls}")
    print(f"texts: {res_chat.text}")
    tool_results = []
    for call in res_chat.tool_calls:
        tool_name = call.name
        tool_params = call.parameters
        function_to_call = tool_to_function_map[tool_name]
        tool_result = {"call": call, "outputs": [function_to_call(**tool_params)]}
        print(f"tool_result: {tool_result}")
        tool_results.append(tool_result)

    chat_detail = ChatDetails(
        chat_request=CohereChatRequest(
            message="",
            tools=tools,
            tool_results=tool_results,
            max_tokens=500,
            chat_history=res_chat.chat_history,
            is_force_single_step=False,
            ),
        compartment_id=OCI_COMPARTMENT_ID,
        serving_mode=OnDemandServingMode(
            model_id=model
        ))

    chat_response = generative_ai_inference_client.chat(chat_detail)
    res_chat = chat_response.data.chat_response

print(res_chat.text)

tool_calls: [{
  "name": "get_employee_id",
  "parameters": {
    "employee_name": "\u7530\u4e2d \u592a\u90ce"
  }
}]
texts: 田中太郎の社員IDを調べ、それから社員詳細情報を取得し、部署のマネージャーを調べる。
tool_result: {'call': {
  "name": "get_employee_id",
  "parameters": {
    "employee_name": "\u7530\u4e2d \u592a\u90ce"
  }
}, 'outputs': [{'employee_name': '田中 太郎', 'employee_id': '12340001'}]}
tool_calls: [{
  "name": "get_employee_detail",
  "parameters": {
    "employee_id": "12340001"
  }
}]
texts: 田中太郎の社員IDは12340001。
tool_result: {'call': {
  "name": "get_employee_detail",
  "parameters": {
    "employee_id": "12340001"
  }
}, 'outputs': [{'employee_id': '12340001', 'employee_detail': {'salary': '¥500', 'department': 'Sales'}}]}
tool_calls: [{
  "name": "get_employee_manager",
  "parameters": {
    "employee_department": "Sales"
  }
}]
texts: 田中太郎の部署はSales。
tool_result: {'call': {
  "name": "get_employee_manager",
  "parameters": {
    "employee_department": "Sales"
  }
}, 'outputs': [{'employee_department': 'Sales

## Book Search

In [11]:
book_themes = {
    '総記': 0,
    '哲学': 1,
    '歴史': 2,
    '社会科学': 3,
    '自然科学': 4,
    '技術': 5,
    '産業': 6,
    '芸術': 7,
    '言語': 8,
    '文学': 9
}

In [12]:
def get_ndc(book_theme: str) -> dict:
    return {"book_theme": book_theme, "book_ndc": book_themes[book_theme]}

def search_books(keyword, ndc):
    API_Endpoint = 'https://ndlsearch.ndl.go.jp/api/opensearch?cnt=10&title=' + keyword + '&ndc=' + str(ndc) + '&dpid=iss-ndl-opac'
    result = requests.get(API_Endpoint)
    return {"book_title": keyword, "book_ndc": ndc, "book_titles": result.text}

In [13]:
tools=[CohereTool(
    name="get_ndc",
    description="「テーマ」(book_theme)から「本の分類」(ndc)を取得する関数",
    parameter_definitions={
            "book_theme": CohereParameterDefinition(
                description="「テーマ」(book_theme)から「本の分類」(ndc)を取得する。例: 哲学",
                type="str",
                is_required=True
            )
        }
    ),
    CohereTool(
    name="search_books",
    description="「キーワード」(keyword)と「本の分類」(ndc)から「本のタイトル」(employee_detail)を取得する関数",
    parameter_definitions={
            "keyword": CohereParameterDefinition(
                description="「キーワード」(keyword)から「本のタイトル」(employee_detail)を取得する。例: マリーアントワネット",
                type="str",
                is_required=True
            ),
            "ndc": CohereParameterDefinition(
                description="「本の分類」(ndc)から「本のタイトル」(employee_detail)を取得する。[0-9]{1}で指定する。例: 0",
                type="str",
                is_required=True
            )
        }
    ),
]

tool_to_function_map = {
    "get_ndc": get_ndc,
    "search_books": search_books,
}

In [14]:
preamble = """
## 回答のスタイル
日本語で回答してください。
質問に対してできる限り詳細な回答をしてください。
"""
input_text = "歴史をテーマにした、織田信長の本を検索して"

In [15]:
chat_detail = ChatDetails(
    chat_request=CohereChatRequest(
        preamble_override=preamble,
        message=input_text,
        tools=tools,
        max_tokens=500,
        is_force_single_step=False,
        ),
    compartment_id=OCI_COMPARTMENT_ID,
    serving_mode=OnDemandServingMode(
        model_id=model
    ))
chat_response = generative_ai_inference_client.chat(chat_detail)
res_chat = chat_response.data.chat_response

while res_chat.tool_calls:
    print(f"tool_calls: {res_chat.tool_calls}")
    print(f"texts: {res_chat.text}")
    tool_results = []
    for call in res_chat.tool_calls:
        tool_name = call.name
        tool_params = call.parameters
        function_to_call = tool_to_function_map[tool_name]
        tool_result = {"call": call, "outputs": [function_to_call(**tool_params)]}
        print(f"tool_result: {tool_result}")
        tool_results.append(tool_result)

    chat_detail = ChatDetails(
        chat_request=CohereChatRequest(
            message="",
            tools=tools,
            tool_results=tool_results,
            max_tokens=500,
            chat_history=res_chat.chat_history,
            is_force_single_step=False,
            ),
        compartment_id=OCI_COMPARTMENT_ID,
        serving_mode=OnDemandServingMode(
            model_id=model
        ))

    chat_response = generative_ai_inference_client.chat(chat_detail)
    res_chat = chat_response.data.chat_response

print(res_chat.text)

tool_calls: [{
  "name": "get_ndc",
  "parameters": {
    "book_theme": "\u6b74\u53f2"
  }
}]
texts: 「歴史」から「本の分類」を取得し、その分類と「織田信長」から「本のタイトル」を取得する。
tool_result: {'call': {
  "name": "get_ndc",
  "parameters": {
    "book_theme": "\u6b74\u53f2"
  }
}, 'outputs': [{'book_theme': '歴史', 'book_ndc': 2}]}
tool_calls: [{
  "name": "search_books",
  "parameters": {
    "keyword": "\u7e54\u7530\u4fe1\u9577",
    "ndc": "2"
  }
}]
texts: 「歴史」の「本の分類」は「2」だった。この分類と「織田信長」から「本のタイトル」を取得する。
tool_result: {'call': {
  "name": "search_books",
  "parameters": {
    "keyword": "\u7e54\u7530\u4fe1\u9577",
    "ndc": "2"
  }
}, 'outputs': [{'book_title': '織田信長', 'book_ndc': '2', 'book_titles': '<rss xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:openSearch="http://a9.com/-/spec/opensearchrss/1.0/" xmlns:dcndl="http://ndl.go.jp/dcndl/terms/" xmlns:dcmitype="http://purl.org/dc/dcmitype/" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:rdfs="http://www.w3.or